In [1]:
import os
from glob import glob
import numpy as np
import json

import cv2
from tqdm.notebook import tqdm

import ffmpeg_utils
from augment import Augmentations
from writer import COCO_writer
from bbox_utils import get_scale_ratio, resize_by_max_side

### Preprocess effects

In [2]:
raw_png_effects_path = 'effects/raw_fire_images'
prep_png_effects_path = 'effects/prep_fire_images'
raw_mov_effects_path = 'effects/raw_fire_vid'
prep_mov_effects_path = 'effects/prep_fire_vid'

vid_out_path = 'output/out.mp4'
annot_out_path = 'output/annotations/instances_default.json'

In [3]:
# Rename and trim empty pixels
for i, effect_path in enumerate(glob(os.path.join(raw_png_effects_path, '*.png'))):
    e_img = cv2.imread(effect_path, cv2.IMREAD_UNCHANGED)
    # Trim empty pixels
    y, x = e_img[:, :, 3].nonzero()
    minx, miny = np.min(x), np.min(y)
    maxx, maxy = np.max(x), np.max(y)
    e_img = e_img[miny:maxy, minx:maxx]
    # Resize to 512px
    scale_ratio = get_scale_ratio(e_img, 512)
    e_img = resize_by_max_side(e_img, scale_ratio)
    cv2.imwrite(os.path.join(prep_png_effects_path, f'fire-{i}.png'), e_img)

In [4]:
# Rename videos
for i, mov_path in enumerate(glob(os.path.join(raw_mov_effects_path, '*.mov'))):
    path = os.path.split(mov_path)[0]
    os.rename(mov_path, os.path.join(path, f'fire-{i}.mov'))

In [5]:
# Resize and extract alpha videos

# for input_path in glob(os.path.join(raw_mov_effects_path, '*.mov')):
#     input_path = os.path.abspath(input_path)
#     filename = os.path.splitext(os.path.split(input_path)[1])[0]
#     out_path = os.path.join(prep_mov_effects_path, filename + '.webm')
#     out_path = os.path.abspath(out_path)
#     ffmpeg_utils.convert_mov2webm(input_path, out_path)
#     ffmpeg_utils.extract_alpha(out_path)

In [6]:
png_effects = glob(os.path.join(prep_png_effects_path, '*.png'))
mov_effects = glob(os.path.join(prep_mov_effects_path, '*.webm'))
png_effects, mov_effects

(['effects/prep_fire_images\\fire-0.png',
  'effects/prep_fire_images\\fire-1.png',
  'effects/prep_fire_images\\fire-2.png'],
 ['effects/prep_fire_vid\\fire-0.webm',
  'effects/prep_fire_vid\\fire-1.webm',
  'effects/prep_fire_vid\\fire-2.webm',
  'effects/prep_fire_vid\\fire-3.webm',
  'effects/prep_fire_vid\\fire-4.webm',
  'effects/prep_fire_vid\\fire-5.webm',
  'effects/prep_fire_vid\\fire-6.webm',
  'effects/prep_fire_vid\\fire-7.webm'])

### Augment

In [7]:
source_videos = glob('source_videos/*')
source_videos

['source_videos\\2020-06-23_16-40-40.mp4',
 'source_videos\\2020-06-23_17-40-41.mp4',
 'source_videos\\stream_OV1_2020-08-01_09_58_11.ts.mp4',
 'source_videos\\stream_OV1_2020-08-01_09_58_50.ts.mp4',
 'source_videos\\stream_OV1_2020-08-01_10_00_12.ts.mp4',
 'source_videos\\stream_OV1_2020-08-01_10_00_39.ts.mp4',
 'source_videos\\stream_OV1_2020-08-01_10_01_29.ts.mp4',
 'source_videos\\stream_OV1_2020-08-01_10_01_56.ts.mp4',
 'source_videos\\stream_OV1_2020-08-01_10_02_31.ts.mp4',
 'source_videos\\stream_OV1_2020-08-01_10_03_28.ts.mp4',
 'source_videos\\stream_OV1_2020-08-01_10_04_06.ts.mp4',
 'source_videos\\stream_OV1_2020-08-01_10_04_34.ts.mp4',
 'source_videos\\stream_OV1_2020-08-01_10_05_28.ts.mp4',
 'source_videos\\stream_OV1_2020-08-01_11_59_21.ts.mp4']

In [12]:
stream = cv2.VideoCapture(source_videos[0])

categories = [
    {
        'name': 'Fire',
        'supercategory': '',
        'id': 1,
    },
    {
        'name': 'Smoke',
        'supercategory': '',
        'id': 2,
    },
]

# COCO_writer
coco_writer = COCO_writer(categories)

# Fire augmentations
fire_augment = Augmentations(png_effects,
                             mov_effects,
                             cat_id=coco_writer.get_cat_id('Fire'),
                             do_resize=True,
                             do_flip=True,
                             do_rotate=False,
                            )

# Create writer
frame_width = int(stream.get(3))
frame_height = int(stream.get(4))
total_frames = int(stream.get(7))
out_stream = cv2.VideoWriter(vid_out_path, cv2.VideoWriter_fourcc(*'MPEG'), 24, (frame_width, frame_height))

pbar = tqdm(total=total_frames)
frame_num = 0
try:
    while stream.isOpened():
        _, frame = stream.read()
        if frame is None:
            print("No image in the stream, stopping.")
            break

        image_name = f'image_{10:06d}.jpg'
        coco_writer.add_frame(*frame.shape[:2], image_name)
        frame = fire_augment.augment(frame, frame_num, coco_writer)
        out_stream.write(frame)
        pbar.update(1)

        frame = cv2.resize(frame, (720, 480))
        cv2.imshow('annotated', frame)
        # Exit on key `q`.
        frame_num += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except KeyboardInterrupt:
    print('Exited.')
finally:
    # Write annotations.
    os.makedirs(os.path.split(annot_out_path)[0], exist_ok=True)
    coco_writer.write_result(annot_out_path)
    # Close streams.
    stream.release()
    out_stream.release()
    cv2.destroyAllWindows()

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


Exited.


#### TODO


##### General
* написать loop чтобы обработать все видосы

##### Preparation
* <s>remove clear pixels</s>


##### Augmentation
* <s>cover all image</s>
* <s>change sizes</s>
* <s>flip</s>
* <s>add animations</s>
* <s>proper resizing</s>
* <s>make an offset point a down center point of the effect image.</s>
* warp perspective
* <s>fix merging with video - fix: Color keying</s>
* <s>change angle</s>
* fix change of angle (maybe polygon annotation?)
* поиграться с color keying'ом, чтобы более плавные переходы в нём были

##### Annotation
* <s>add dynamic bboxes for videos</s>
* <s>fix bboxes for some videos</s>
* <s>annotate all videos</s>
* <s>annotation for images</s>
* <s>scaling bboxes</s>
* <s>rotating bboxes</s>
* <s>Write to COCO</s>
* разметить эффекты не боксом, а полигоном, чтобы можно было нормально поворачивать эффект и при этом иметь нормальный бокс



## Working with videos

#### Scale down with keeping alpha channel
```shell
ffmpeg -i in.mov -filter:v scale=720:-1 -c:v qtrle out.mov
```

#### `.mov` to `.webm` keeping  alpha channel

```shell
ffmpeg -i in.mov -c:v libvpx -pix_fmt yuva420p -auto-alt-ref 0 out.webm
```

#### Extract alpha channel from `.webm`

```shell
ffmpeg -vcodec libvpx -i in.webm -vf alphaextract -y out.mp4
```


#### Extract alpha channel from `.mov`

```shell
ffmpeg -i in.mov -vf alphaextract,format=yuv420p out.mp4
```

#### Compress resulting video
```shell
ffmpeg -i out.mp4 -vcodec h264 -b:v 1000k -acodec mp2 compressed_out.mp4 -y
```